# Content based filtering

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
movie = pd.read_csv('movies_metadata.csv')

C:\Users\EonKim\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
#Read in the data
df = movie[:31000]
print(df.shape)
df.head()

(31000, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
df.describe() # num_col = ['revenue', 'runtime', 'vote_average', 'vote_count']

,revenue,runtime,vote_average,vote_count
count,3.099600e+04,30912.000000,30996.000000,30996.000000
mean,1.493675e+07,97.307130,5.776065,143.232256
std,7.362557e+07,34.235179,1.725090,569.236382
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,87.000000,5.200000,4.000000
50%,0.000000e+00,96.000000,6.100000,14.000000
75%,0.000000e+00,108.000000,6.800000,52.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31000 entries, 0 to 30999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  31000 non-null  object 
 1   belongs_to_collection  3469 non-null   object 
 2   budget                 31000 non-null  object 
 3   genres                 31000 non-null  object 
 4   homepage               5044 non-null   object 
 5   id                     31000 non-null  object 
 6   imdb_id                30988 non-null  object 
 7   original_language      30997 non-null  object 
 8   original_title         31000 non-null  object 
 9   overview               30626 non-null  object 
 10  popularity             30996 non-null  object 
 11  poster_path            30831 non-null  object 
 12  production_companies   30998 non-null  object 
 13  production_countries   30998 non-null  object 
 14  release_date           30967 non-null  object 
 15  re

In [14]:
df.isna().sum()

adult                        0
belongs_to_collection    27531
budget                       0
genres                       0
homepage                 25956
id                           0
imdb_id                     12
original_language            3
original_title               0
overview                   374
popularity                   4
poster_path                169
production_companies         2
production_countries         2
release_date                33
revenue                      4
runtime                     88
spoken_languages             4
status                      49
tagline                  14787
title                        4
video                        4
vote_average                 4
vote_count                   4
dtype: int64

In [15]:
df['overview']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
30995    Spoof of the "Taken" franchise. An ex-CIA agen...
30996    In the near future the earth has become desola...
30997    Reincarnation goes horribly wrong releasing da...
30998    Seven characters, introduced at the start of t...
30999    After the Ball, a retail fairy tale set in the...
Name: overview, Length: 31000, dtype: object

In [17]:
#Using TF-IDF vectorizer

tfidf = TfidfVectorizer(stop_words = 'english')
df['overview'] = df['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['overview'])
print(tfidf_matrix.shape)
tfidf_matrix

<ipython-input-17-b637f7ffff5b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview'] = df['overview'].fillna('')
(31000, 59830)


<31000x59830 sparse matrix of type '<class 'numpy.float64'>'
	with 828587 stored elements in Compressed Sparse Row format>

In [23]:
# Cosine Similarity based distance matrix 
cos = linear_kernel(tfidf_matrix, tfidf_matrix)
print('cos.shape :',cos.shape)
cos

cos.shape : (31000, 31000)


array([[1.        , 0.01556263, 0.        , ..., 0.        , 0.01655058,
        0.        ],
       [0.01556263, 1.        , 0.04873332, ..., 0.        , 0.01982613,
        0.02658255],
       [0.        , 0.04873332, 1.        , ..., 0.01017753, 0.        ,
        0.00434099],
       ...,
       [0.        , 0.        , 0.01017753, ..., 1.        , 0.        ,
        0.        ],
       [0.01655058, 0.01982613, 0.        , ..., 0.        , 1.        ,
        0.00374511],
       [0.        , 0.02658255, 0.00434099, ..., 0.        , 0.00374511,
        1.        ]])

In [30]:
# 인덱스 값에 title을 지정해줌
indices = pd.Series(df.index, index = df['title']).drop_duplicates() # 중복제거 
indices.head()

# idx 를 입력하면 거기에 해당하는 0, 1, 2, 이런 값이 출력된다 .

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

Index([                     'Toy Story',                        'Jumanji',
                     'Grumpier Old Men',              'Waiting to Exhale',
          'Father of the Bride Part II',                           'Heat',
                              'Sabrina',                   'Tom and Huck',
                         'Sudden Death',                      'GoldenEye',
       ...
                          'Wild Horses',                   'Search Party',
                           'The Living',             'Maya the Bee Movie',
       'Extinction: Nature Has Evolved',                         'Tooken',
                    'Battle For SkyArk',              'The Scopia Effect',
                              'Party 7',                 'After the Ball'],
      dtype='object', name='title', length=31000)

In [51]:
idx = indices['Iron Man'] # 해당 위치를 찾기 위함
sim_scores = list(enumerate(cos[idx])) # 해당 idx = 영화와 각 영화들 간의 유사도를 계산
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # 유사도 기준으로 sort해준다. 큰 순서대로 
sim_scores = sim_scores[1:6] # 본인 영화를 제외한 top 5를 추천해준다. 

In [99]:
(np.round(sim, 3) * 100) 

array([18.4, 18.3, 18.1, 15. , 14.4])

In [112]:
sim_scores

[(7555, 0.18428462420789665),
 (15153, 0.18291269144034386),
 (28654, 0.18056705766816955),
 (15199, 0.14957211684281596),
 (20830, 0.14395829464820611)]

In [117]:
sim = ["{0:.4f}".format(i[1]) for i in sim_scores]
sim

['0.1843', '0.1829', '0.1806', '0.1496', '0.1440']

In [126]:
def get_recommendations(title, cos = cos ):
    idx = indices[title] # 해당 위치를 찾기 위함
    sim_scores = list(enumerate(cos[idx])) # 해당 idx = 영화와 각 영화들 간의 유사도를 계산
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # 유사도 기준으로 sort해준다. 큰 순서대로 
    sim_scores = sim_scores[1:6] # 본인 영화를 제외한 top 5를 추천해준다. 
    movie_indices = [i[0] for i in sim_scores] # idx에 집어넣어준다. 
    sim = ["{0:.2f} %".format(i[1]*100) for i in sim_scores]
    movies=df['title'].iloc[movie_indices] # 상위 5개 종목의 idx값을 기준으로 뽑아온다 movie이름을
    imdb_id=df['imdb_id'].iloc[movie_indices] # imdb_id도 뽑아온다
    dict={"Movies":movies,"IMDB_id":imdb_id, 'similar(%)' : sim  } # 두 값을 dict로 묶어준다.
    final_df=pd.DataFrame(dict) # dict를 DataFrame으로 변환
    final_df.reset_index(drop=True,inplace=True) # idx를 새로 만들어줌
    return final_df # return 

In [127]:
get_recommendations('Iron Man')

,Movies,IMDB_id,similar(%)
0,The Wonderful Ice Cream Suit,tt0129634,18.43 %
1,Iron Man 2,tt1228705,18.29 %
2,Clown,tt1780798,18.06 %
3,The Disappearance of Alice Creed,tt1379177,14.96 %
4,Iron Man 3,tt1300854,14.40 %


In [129]:
get_recommendations('Clown')

,Movies,IMDB_id,similar(%)
0,The Wonderful Ice Cream Suit,tt0129634,41.51 %
1,P.U.N.K.S,tt0130172,31.61 %
2,A Wedding Suit,tt0074782,27.91 %
3,Gorilla at Large,tt0047041,21.43 %
4,Iron Man,tt0371746,18.06 %
